In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# https://stackoverflow.com/questions/66828031/do-i-always-have-to-restart-my-kernel-in-jupyter-lab-when-code-in-a-local-module

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')

# Dataset Generation
The functions to manipulate the dataset allow for

In [72]:
from DatasetFuncs import allDataset_loader,dataset_split,show_porcentages,show_partition_nanopores

data_folder="../ext/QuipuData/"; #path where the datasets are/will be stored
allDatasets=allDataset_loader(data_folder) #Generates the processed df and saves it. If it is already in memory
#then it loads it. One can specify cut=False to obtain the uncut traces (if not they are cut at 700 samples as was used in quipunet)

trainSet,testSet=dataset_split(allDatasets)#Divides in train and test, in a way that the test set is within a range of
# % of samples, and that nanopores used on train set are not used in train for the same barcode.
samples_perc=show_porcentages(trainSet,testSet) #This verifies the percentages

     Train  Test
000   5769   315
001   7665  1340
010   2418   101
011  14231  2378
100    919    83
101   7551   427
110   6731   606
111   6979   665
         Train       Test
000  94.822485   5.177515
001  85.119378  14.880622
010  95.990472   4.009528
011  85.682461  14.317539
100  91.716567   8.283433
101  94.647781   5.352219
110  91.740493   8.259507
111  91.300366   8.699634


In [73]:
#With this code we show which is the nanopores used for this train and test dataset
print(show_partition_nanopores(trainSet))
print(show_partition_nanopores(testSet))

   barcode  nanopore  size
0      000         7  2172
1      000         8  3498
2      000      1017    99
3      001         8  1737
4      001        27  3583
5      001        28   923
6      001        29  1255
7      001      1053   167
8      010         7  1021
9      010         9   629
10     010        10   768
11     011        11   362
12     011        12   577
13     011        31   899
14     011        33  2192
15     011        35   483
16     011        36  1780
17     011        37  1583
18     011        38  1229
19     011        39  1735
20     011        40  2120
21     011        41  1271
22     100         7   288
23     100        13   631
24     101         8  1240
25     101        26  1764
26     101        27   924
27     101        29  3494
28     101      1662   129
29     110         7  1161
30     110         8  2702
31     110        13  1309
32     110        14  1559
33     111         7   735
34     111         8  2866
35     111         9   677
3

# Testing reproducibility

With this function we can run the training of quipu how it was done in their notebook (with n epochs=60)

In [3]:
from ModelTrainer import ModelTrainer
mt=ModelTrainer()
mt.quipu_def_train(n_epochs=2)

/home/jkipen/anaconda3/envs/browDataAug/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


=== Epoch: 0 ===
  prep time: 3.0 sec   train time: 21.0 sec
  loss: 1.893   acc: 0.2963   val_acc: 0.4077
=== Epoch: 1 ===
  prep time: 3.0 sec   train time: 17.0 sec
  loss: 1.694   acc: 0.3553   val_acc: 0.4316
       [ loss , accuracy ]
Train: [1.465266466140747, 0.4333359897136688]
Validation  : [1.4687469005584717, 0.43164435029029846]
Test : [1.507033109664917, 0.49643704295158386]


(0.4333359897136688, 0.43164435029029846, 0.49643704295158386)

Then we did this for 20 runs and saved it in "../results/QuipuReproduction.csv", so when we load the datafram we can observe the accuracies and their variance.

In [12]:
def print_accs(array_accs): #Array that has 3 columns, train valid and test accs, each row is a model result
    all_means=np.mean(all_accs,axis=0)
    all_stds=np.std(all_accs,axis=0)
    print("Train Acc: " + "{:.2f}".format(all_means[0]*100) + " +- " + "{:.2f}".format(all_stds[0]*100) + " % ")
    print("Valid Acc: " + "{:.2f}".format(all_means[1]*100) + " +- " + "{:.2f}".format(all_stds[1]*100) + " % ")
    print("Test Acc: " + "{:.2f}".format(all_means[2]*100) + " +- " + "{:.2f}".format(all_stds[2]*100) + " % ")
    n_samples=np.shape(array_accs)[0]
    print("Test Accuracy estimator: " + "{:.2f}".format(all_means[2]*100) + " +- " + "{:.2f}".format(all_stds[2]*100/np.sqrt(n_samples)) + " % ")
    

In [13]:
df=pd.read_csv("../results/QuipuReproduction.csv")
all_accs=df.iloc[:,:-1].values
print_accs(all_accs)

Train Acc: 97.65 +- 0.27 % 
Valid Acc: 96.20 +- 0.48 % 
Test Acc: 92.84 +- 1.47 % 
Test Accuracy estimator: 92.84 +- 0.33 % 


In [4]:
np.mean(df["Test Acc"].values[np.argsort(df["Test Acc"].values)[-2:]]) #Top 10% test accuracy (2 samples is 10% of 20)

0.9513720571994781

In [14]:
df=pd.read_csv("../results/QuipuTrainedWithES.csv")
all_accs=df.iloc[:,:-1].values
print_accs(all_accs)

Train Acc: 98.90 +- 0.17 % 
Valid Acc: 96.53 +- 0.24 % 
Test Acc: 92.58 +- 1.77 % 
Test Accuracy estimator: 92.58 +- 0.40 % 


## To check if we can remove it. Old used to debug

In [10]:
np.shape(all_accs)[0]

20

In [3]:
from DataLoader import DataLoader
from DataAugmentator import DataAugmentator
dl=DataLoader();
#X_train,X_valid,Y_train,Y_valid,X_test,Y_test=dl.get_datasets_numpy_quipu();
X_train,X_valid,Y_train,Y_valid,X_test,Y_test=dl.get_datasets_numpy();
da=DataAugmentator();
out=da.test_brow_aug(X_train);

<class 'TypeError'>: 'tuple' object cannot be interpreted as an integer